<a href="https://colab.research.google.com/github/PaoloBPP/Projeto_Avalia-o_Localizador/blob/main/Localizador_Pessoas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [287]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [288]:
#Configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('chave_secreta')
genai.configure(api_key=api_key)

In [ ]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [290]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [292]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [293]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

Este código gera os dados pra ser digitados sendo que o sexo e status é selecionado por um menu de opções

In [ ]:
chat = model.start_chat(history=[])

while prompt != "fim":
    # Input fields
    nome = input("Digite seu nome  (ou 'fim' para parar): ")
    if nome == "fim":
        break

    idade = int(input("Digite sua idade: "))
    cpf = input("Digite seu CPF: ")
    endereco = input("Digite seu endereço: ")
    telefone = input("Digite seu telefone: ")
     # Gender selection using menu
    while True:
        print("\nSelecione seu gênero:")
        print("1. Masculino")
        print("2. Feminino")
        print("3. Infantil")

        escolha_genero = input("Digite sua escolha (1, 2 ou 3): ")

        if escolha_genero in ["1", "2", "3"]:
            if escolha_genero == "1":
                genero = "Masculino"
            elif escolha_genero == "2":
                genero = "Feminino"
            else:
                genero = "Infantil"
            break
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")
     # Gender selection using menu
    while True:
        print("\nSelecione seu status:")
        print("1. Encontrado")
        print("2. Alojado")
        print("3. Desaparecido")

        escolha_status = input("Digite sua escolha (1, 2 ou 3): ")

        if escolha_status in ["1", "2", "3"]:
            if escolha_status == "1":
                status = "Encontrado"
            elif escolha_status == "2":
                status = "Alojado"
            else:
                status = "Desaparecido"
            break
        else:
            print("Opção inválida. Digite 1, 2 ou 3.")




    # Displaying user input
    print("Nome:", nome)
    print("Idade:", idade)
    print("CPF:", cpf)
    print("Telefone:", telefone)
    print("Endereço:", endereco)
    print("Sexo:", genero)
    print("Sttus:", status)

    # Constructing a prompt with user inputs
    prompt = f"Olá, meu nome é {nome}, tenho {idade} anos, meu CPF é {cpf}, meu Numero telefome é {telefone}, moro em {endereco}, sou {genero} e estou {status}."

    # Sending prompt to the model
    response = chat.send_message(prompt)


In [ ]:
#Melhorando a visualização
#Código disponível em https://ai.google.dev/tutorials/python_quickstart#import_packages
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#Imprimindo o histórico
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
  print('-------------------------------------------')

Este código serve como um filtro buscar algo que queira como por exemplo se a pessoa foi cadastrada
Um exemplo de pesquisa
(Encontrado, Desaparecido, Alojado)

In [ ]:
import re
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def search_filter(keyword):
    """
    Aplica um filtro de pesquisa no histórico de mensagens.

    Args:
        keyword (str): A palavra-chave a ser pesquisada.

    Returns:
        list: Uma lista contendo as mensagens que correspondem à pesquisa.
    """
    filtered_messages = []
    for message in chat.history:
        if re.search(keyword, message.parts[0].text, re.IGNORECASE):
            filtered_messages.append(message)
    return filtered_messages

# Imprimindo o histórico com filtro
keyword = input("Digite a palavra-chave para pesquisa: ")
filtered_messages = search_filter(keyword)

if filtered_messages:
    for message in filtered_messages:
        display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))
        print('-------------------------------------------')
else:
    print(f"Nenhuma mensagem encontrada para a palavra-chave '{keyword}'.")

**Serve pra incluir uma imagem ao nome da Pessoa que ira mostrar em arquivos**

In [ ]:
from google.colab import files
from PIL import Image
from PIL import ImageOps  # Import for resize functionality
import io

uploaded = files.upload()

# Define base resize factor (modify as needed)
base_resize_factor = 0.5  # Reduce image size by 50% initially

for filename, file_content in uploaded.items():
    # Get size using len()
    file_size = len(file_content)
    print(f"Nome do arquivo original: {filename}")
    print(f"Tamanho do arquivo: {file_size} bytes")

    try:
        # Attempt to open the image using a broader mode
        image_data = io.BytesIO(file_content)
        image = Image.open(image_data, 'r')  # Use 'r' for read mode

        width, height = image.size

        # Calculate a dynamic resize factor based on desired maximum dimensions
        # (modify max_width and max_height as needed)
        max_width = 300  # Set your desired maximum width for display
        max_height = 200  # Set your desired maximum height for display
        resize_factor = min(max_width / width, max_height / height)

        # Ensure resize factor is at least base_resize_factor (optional)
        resize_factor = max(resize_factor, base_resize_factor)

        # Get user input for new filename (without extension)
        new_filename = input("Digite o novo nome para o arquivo (sem extensão): ")

        # Construct the new filename with extension (modify extension as needed)
        new_filename_with_ext = f"{new_filename}.jpg"

        # Resize the image
        resized_image = image.resize((int(width * resize_factor), int(height * resize_factor)), Image.ANTIALIAS)

        # Display the resized image
        display(resized_image)

        # Save the resized image with the new filename (optional)
        with open(new_filename_with_ext, 'wb') as f:
            resized_image.save(f, format='JPEG')  # Save as JPEG (modify format as needed)
            print(f"Imagem salva como: {new_filename_with_ext}")

    except (IOError, OSError) as e:
        print(f"Error opening file '{filename}': {e}")